In [1]:
from scipy import misc, ndimage, signal                                        
import numpy as np                                                              
import random                                                                  
#import ntpath                                                                   
import os                                                                      
import pandas as pd                                                            
import matplotlib as mpl                                                        
import matplotlib.pyplot as plt                                                 
import matplotlib.colors as colors                                             
import tensorflow as tf                                                         
                                                                                
from tensorflow.keras.datasets import mnist
from tensorflow.keras import optimizers                                         
from tensorflow.keras import regularizers                                      
from tensorflow.keras import backend as K                                       
from tensorflow.keras import datasets,layers,models,Input,Model
from tensorflow.keras.optimizers import RMSprop, Adam, Adagrad, SGD, Adadelta, Adamax, Nadam
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, AveragePooling2D, Cropping2D
from tensorflow.keras.layers import Dropout, Activation, Flatten, Concatenate, Dense, Reshape, Add, PReLU, LeakyReLU, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.keras.activations import relu
import time as tm                                                               
from time import time                                                           
import datetime                                                                 
from operator import itemgetter                                                
                                                                                
import glob                                                                     
import tensorflow.keras.utils                                                   
from tensorflow.keras.utils import to_categorical                               
from numpy import argmax                                                       
import seaborn as sn                                                           
from sklearn.model_selection  import train_test_split                           
                                                                                
from sklearn.metrics import confusion_matrix                                    
from sklearn.metrics import accuracy_score                                      
from sklearn.metrics import f1_score                                            
from sklearn.metrics import recall_score                                        
from sklearn.metrics import precision_score                                     
from sklearn.metrics import classification_report                               
from tensorflow.keras import layers                                            
from sklearn.model_selection import GridSearchCV
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# **2. To load data**

In [3]:
X = np.load('/home/bioml/Projects/PhD/InpactorDB/version_final/InpactorDB_non-redudant.fasta.filtered_zeros.npy').astype(bool)

print(X.shape)

(67305, 5, 24902)


# **3. Model definitions**

## **3.1 K-mer counting**

In [9]:
def CONTEO_KMER(X, optimizador=Adam,lr=0.001,momen=0,init_mode='glorot_uniform',fun_act='relu',dp=0.2,regularizer=l2,w_reg=0):
    tf.keras.backend.clear_session()
    
    k=0
    n_1=4**1
    W_1=np.zeros((5,1,1,n_1))
    b_1=np.zeros((1,n_1)).reshape(n_1,)
    for i in range(4):
      W_1[i,0,0,i]=1
    
    k=0
    n_2=4**2
    W_2=np.zeros((5,2,1,n_2))
    b_2=-np.ones((1,n_2)).reshape(n_2,)
    for i in range(4):
      for j in range(4):
        W_2[i,0,0,k]=1
        W_2[j,1,0,k]=1
        k=k+1

    k=0
    n_3=4**3
    W_3=np.zeros((5,3,1,n_3))
    b_3=-2*np.ones((1,n_3)).reshape(n_3,)
    for i in range(4):
      for j in range(4):
        for l in range(4):
          W_3[i,0,0,k]=1
          W_3[j,1,0,k]=1
          W_3[l,2,0,k]=1
          k=k+1

    k=0
    n_4=4**4
    W_4=np.zeros((5,4,1,n_4))
    b_4=-3*np.ones((1,n_4)).reshape(n_4,)
    for i in range(4):
      for j in range(4):
        for l in range(4):
          for m in range(4):
            W_4[i,0,0,k]=1
            W_4[j,1,0,k]=1
            W_4[l,2,0,k]=1
            W_4[m,3,0,k]=1
            k=k+1

    k=0
    n_5=4**5
    W_5=np.zeros((5,5,1,n_5))
    b_5=-4*np.ones((1,n_5)).reshape(n_5,)
    for i in range(4):
      for j in range(4):
        for l in range(4):
          for m in range(4):
            for n in range(4):
              W_5[i,0,0,k]=1
              W_5[j,1,0,k]=1
              W_5[l,2,0,k]=1
              W_5[m,3,0,k]=1
              W_5[n,4,0,k]=1
              k=k+1

    k=0
    n_6=4**6
    W_6=np.zeros((5,6,1,n_6))
    b_6=-5*np.ones((1,n_6)).reshape(n_6,)
    for i in range(4):
      for j in range(4):
        for l in range(4):
          for m in range(4):
            for n in range(4):
              for p in range(4):
                W_6[i,0,0,k]=1
                W_6[j,1,0,k]=1
                W_6[l,2,0,k]=1
                W_6[m,3,0,k]=1
                W_6[n,4,0,k]=1
                W_6[p,5,0,k]=1
                k=k+1

    # Inputs
    inputs = tf.keras.Input(shape=(X.shape[1], X.shape[2], 1), name="input_1")
    # layer 1
    
    layers_1 = tf.keras.layers.Conv2D(n_1, (5, 1), strides=(1,1), weights=[W_1,b_1],activation=fun_act, use_bias=True, name='Mental_01')(inputs)
    layers_1=tf.keras.backend.sum(layers_1,axis=-2)

    layers_2 = tf.keras.layers.Conv2D(n_2, (5, 2), strides=(1,1), weights=[W_2,b_2],activation=fun_act, use_bias=True, name='Mental_02')(inputs)
    layers_2=tf.keras.backend.sum(layers_2,axis=-2)

    layers_3 = tf.keras.layers.Conv2D(n_3, (5, 3), strides=(1,1), weights=[W_3,b_3],activation=fun_act, use_bias=True, name='Mental_03')(inputs)
    layers_3 = tf.keras.backend.sum(layers_3,axis=-2)

    layers_4 = tf.keras.layers.Conv2D(n_4, (5, 4), strides=(1,1), weights=[W_4,b_4],activation=fun_act, use_bias=True, name='Mental_04')(inputs)
    layers_4 = tf.keras.backend.sum(layers_4,axis=-2)

    layers_5 = tf.keras.layers.Conv2D(n_5, (5, 5), strides=(1,1), weights=[W_5,b_5],activation=fun_act, use_bias=True, name='Mental_05')(inputs)
    layers_5 = tf.keras.backend.sum(layers_5,axis=-2)

    layers_6 = tf.keras.layers.Conv2D(n_6, (5, 6), strides=(1,1), weights=[W_6,b_6],activation=fun_act, use_bias=True, name='Mental_06')(inputs)
    layers_6 = tf.keras.backend.sum(layers_6,axis=-2)

    layers=tf.concat([layers_1,layers_2,layers_3, layers_4,layers_5, layers_6],2)
    layers = tf.keras.layers.Flatten()(layers)
    model = tf.keras.Model(inputs = inputs, outputs=layers)

    opt = optimizador(learning_rate=lr)
    # loss function
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    # Compile model
    #model.compile(loss=loss_fn, optimizer=opt, metrics=['acc', 'AUC', 'mse','mae','mape'])
    model.compile(loss=loss_fn, optimizer=opt)
    return model

model_name='CONTEO_KMER'
model = CONTEO_KMER(X)
print(model.summary())

counts=model.predict(X, batch_size=2)
print('counts dims',counts.shape)
print('counts = ',counts)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5, 24902, 1) 0                                            
__________________________________________________________________________________________________
Mental_01 (Conv2D)              (None, 1, 24902, 4)  24          input_1[0][0]                    
__________________________________________________________________________________________________
Mental_02 (Conv2D)              (None, 1, 24901, 16) 176         input_1[0][0]                    
__________________________________________________________________________________________________
Mental_03 (Conv2D)              (None, 1, 24900, 64) 1024        input_1[0][0]                    
______________________________________________________________________________________________

In [16]:
weights = CONTEO_KMER()
print(weights)
save('Weights_SL.npy', weights.astype(np.int8))

[array([[[[1., 0., 0., 0.]]],


       [[[0., 1., 0., 0.]]],


       [[[0., 0., 1., 0.]]],


       [[[0., 0., 0., 1.]]],


       [[[0., 0., 0., 0.]]]])
 array([0., 0., 0., 0.])
 array([[[[1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]],

        [[1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
          0.]]],


       [[[0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
          0.]],

        [[0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
          0.]]],


       [[[0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
          0.]],

        [[0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
          0.]]],


       [[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
          1.]],

        [[0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
          1.]]],


       [[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]],

        [[0., 0., 0., 0., 0.

/home/bioml/anaconda3/envs/tf22/lib/python3.7/site-packages/ipykernel_launcher.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


NameError: name 'save' is not defined